In [ ]:
!pip install openai pydantic

In [5]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",  # OpenRouter OpenAI-compatible base URL
)

In [19]:
# make posts from doc
from typing import Dict, Any, List

import requests
import unicodedata


# retrieve data from notion

NOTION_TOKEN = userdata.get("NOTION_API")
PAGE_ID = "2ee0a735274f80b6a961ee81eed44b06"
NOTION_VERSION = "2025-09-03"

headers = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Notion-Version": NOTION_VERSION,
    "Content-Type": "application/json",
}

def notion_get(url: str, params: Dict[str, Any] | None = None) -> Dict[str, Any]:
    r = requests.get(url, headers=headers, params=params)
    r.raise_for_status()
    return r.json()

def list_block_children(block_id: str) -> List[Dict[str, Any]]:
    """Fetches all child blocks (paginated)."""
    results = []
    url = f"https://api.notion.com/v1/blocks/{block_id}/children"
    cursor = None
    while True:
        params = {"start_cursor": cursor} if cursor else None
        data = notion_get(url, params=params)
        results.extend(data.get("results", []))
        if not data.get("has_more"):
            break
        cursor = data.get("next_cursor")
    return results

def rich_text_to_plain(rt: List[Dict[str, Any]]) -> str:
    return "".join(x.get("plain_text", "") for x in (rt or []))

def block_to_text(block: Dict[str, Any]) -> str:
    t = block["type"]
    if t in ("paragraph", "heading_1", "heading_2", "heading_3", "bulleted_list_item", "numbered_list_item", "to_do", "quote", "callout"):
        obj = block[t]
        return rich_text_to_plain(obj.get("rich_text", []))
    if t == "code":
        obj = block[t]
        lang = obj.get("language", "")
        code = rich_text_to_plain(obj.get("rich_text", []))
        return f"```{lang}\n{code}\n```"
    if t == "divider":
        return "---"
    if t == "image":
        return block["image"].get("caption", [{}])[0].get("plain_text", "") or "[image]"
    return f"[{t}]"
def normalize(text: str)-> str:
    return unicodedata.normalize("NFKD", text)

def walk_blocks(block_id: str, indent: int = 0) -> List[str]:
    lines = []
    children = list_block_children(block_id)
    for b in children:
        prefix = "  " * indent
        text = block_to_text(b)
        if text.strip():
            lines.append(prefix + text)
        if b.get("has_children"):
            lines.extend(walk_blocks(b["id"], indent=indent + 1))
    return normalize("\n".join(lines))




lines = walk_blocks(PAGE_ID)







A company that gives oral exams based on material uploaded. 
Description:
A platform that automatically generates and administers oral examinations based on user-uploaded materials. Educators, institu


In [31]:
from datetime import date

In [40]:
# make posts from llm
from datetime import date
from pydantic import BaseModel, Field, ValidationError, field_validator, model_validator

class Status(BaseModel):
    status: str = Field(min_length=3, description="The post text to publish on Mastodon (<= 100 words).")
    date_posted: date = Field(description="Target date for posting (YYYY-MM-DD).")
    content: str = Field(description="Short internal rationale or angle for the post (not posted).")

class StatusList(BaseModel):
    posts: List[Status] = Field(min_length=1, description="List of generated posts.")


num_posts = 5
instructions = f"make {num_posts} post markting my company under 100 words using the following information: " + lines
posts = client.responses.parse(
      model="nvidia/nemotron-3-nano-30b-a3b:free",
      input= instructions,
      text_format=StatusList,
  )
# posts

ParsedResponse[StatusList](id='gen-1768966354-hcIXByFWNGW3kf0SM2aA', created_at=1768966354.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='nvidia/nemotron-3-nano-30b-a3b:free', object='response', output=[ResponseReasoningItem(id='rs_tmp_ar49c92fovw', summary=[], type='reasoning', content=[Content(text="We need to produce 5 promotional posts under 100 words each, based on given info. Provide 5 separate posts, each <100 words. Should be marketing style. We'll deliver concise posts.\n\nWe'll count roughly. Provide them as separate bullet points or numbered. Ensure each is within 100 words.\n\nLet's craft.\n", type='reasoning_text')], encrypted_content=None, status=None), ParsedResponseOutputMessage[StatusList](id='msg_tmp_sz9l2xm508', content=[ParsedResponseOutputText[StatusList](annotations=[], text='{\n "posts": [\n   {\n     "status": "Ready",\n     "date_posted": "2025-11-02",\n     "content": "🚀 Transform exams with AI‑powered oral assessments! Our plat

In [46]:
import requests

parsed = (
    getattr(posts, "output_parsed", None)
    or getattr(posts, "parsed", None)
    or posts
)

hashtag = " #marketing #AIGenerated"  # note leading space

MASTODON_INSTANCE = "https://mastodon.social"
ACCESS_TOKEN = userdata.get("MASTODON_API")
url = f"{MASTODON_INSTANCE}/api/v1/statuses"

headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
}

def make_post():
    for post in parsed.posts:
        payload = {
            "status": post.content + hashtag,  # FIXED
            "visibility": "unlisted",
        }

        r = requests.post(url, data=payload, headers=headers)
        r.raise_for_status()

        print("Posted:", r.json()["url"])

make_post()


Posted: https://mastodon.social/@cesia/115931058068552336
Posted: https://mastodon.social/@cesia/115931058110818092
Posted: https://mastodon.social/@cesia/115931058152382544
Posted: https://mastodon.social/@cesia/115931058212836028
Posted: https://mastodon.social/@cesia/115931058280759310


In [47]:
from bs4 import BeautifulSoup
from typing import List, Optional
from pydantic import BaseModel, Field

def html_to_text(html: str) -> str:
    return BeautifulSoup(html or "", "html.parser").get_text(" ", strip=True)

def mastodon_search_statuses(instance: str, token: str, keyword: str, limit: int = 5):
    """
    Uses Mastodon v2 search endpoint to find statuses.
    Note: Some instances require auth and/or only return results within your instance / context.
    """
    url = f"{instance}/api/v2/search"
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "q": keyword,
        "type": "statuses",
        "limit": limit,
        "resolve": True,  # attempt to resolve remote results (instance-dependent)
    }
    r = requests.get(url, headers=headers, params=params)
    r.raise_for_status()
    data = r.json()
    return data.get("statuses", [])
class PostForReply(BaseModel):
    id: str
    url: Optional[str] = None
    author_acct: str
    text: str

class ReplyDraft(BaseModel):
    in_reply_to_id: str
    reply_text: str = Field(min_length=1, description="Reply text WITHOUT the leading @handle (we add that).")

class ReplyBatch(BaseModel):
    replies: List[ReplyDraft] = Field(min_length=1)


KEYWORD = "#Articulate"

MASTODON_INSTANCE = "https://mastodon.social"
ACCESS_TOKEN = userdata.get("MASTODON_API")  # or your token string

raw_statuses = mastodon_search_statuses(MASTODON_INSTANCE, ACCESS_TOKEN, KEYWORD, limit=5)

posts_for_llm: List[PostForReply] = []
for s in raw_statuses[:5]:
    posts_for_llm.append(
        PostForReply(
            id=s["id"],
            url=s.get("url"),
            author_acct=s["account"]["acct"],
            text=html_to_text(s.get("content", ""))[:1200],  # keep prompt compact
        )
    )

BUSINESS_BLURB = (
    "My company helps educators administer oral exams to students "
)

instructions = f"""
You are writing replies on Mastodon.

Goal:
- Write thoughtful, non-spammy replies to 5 recent posts related to: "{KEYWORD}"
- Replies should be genuinely useful and context-aware.
- Keep each reply <= 60 words.
- Do NOT include links unless the original post asks for resources.
- Avoid over-marketing; at most, a soft mention like "I work on X" if relevant.
- Be polite and human.

Business context:
{BUSINESS_BLURB}

Return JSON matching the schema ReplyBatch.
Each ReplyDraft must reference the exact in_reply_to_id from the provided posts.
"""

# Provide the posts as data to ground the generation
prompt_payload = {
    "instructions": instructions,
    "posts": [p.model_dump() for p in posts_for_llm],
}

response = client.responses.parse(
    model="nvidia/nemotron-3-nano-30b-a3b:free",
    input=str(prompt_payload),
    text_format=ReplyBatch,
)

parsed = getattr(response, "output_parsed", None) or getattr(response, "parsed", None) or response
reply_batch: ReplyBatch = parsed
reply_batch



ReplyBatch(replies=[ReplyDraft(in_reply_to_id='115930893404544962', reply_text='Great insights! Clear, concise speaking is key in any classroom. Our platform helps educators design oral exams that train students to articulate their ideas clearly, building confidence over time. Happy to discuss how we support that process!'), ReplyDraft(in_reply_to_id='115861572553958934', reply_text='Congrats on completing those modules! Storyline is powerful, and we’ve seen educators use similar structured tools to assess verbal skills. Our solution lets you create and evaluate oral exams directly within your LMS, reducing the need for work‑arounds. Let me know if you’d like to explore integration ideas.'), ReplyDraft(in_reply_to_id='115721110562555815', reply_text='Exactly—simple, clear messaging builds trust, whether leading a team or guiding students. Our tools help educators craft precise oral instructions that students can confidently deliver. It’s rewarding to see clearer articulation translate 

In [50]:
def post_replies(instance: str, token: str, posts: List[PostForReply], batch: ReplyBatch, visibility: str = "unlisted"):
    url = f"{instance}/api/v1/statuses"
    headers = {"Authorization": f"Bearer {token}"}

    # Map id -> author acct
    id_to_acct = {p.id: p.author_acct for p in posts}

    posted_urls = []
    for r in batch.replies:
        acct = id_to_acct.get(r.in_reply_to_id)
        if not acct:
            continue

        status_text = f"@{acct} {r.reply_text}".strip()

        payload = {
            "status": status_text,
            "in_reply_to_id": r.in_reply_to_id,
            "visibility": visibility,  # "public" | "unlisted" | "private" | "direct"
        }

        resp = requests.post(url, data=payload, headers=headers)
        resp.raise_for_status()
        posted_urls.append(resp.json().get("url"))
        print("Replied:", posted_urls[-1])

    return posted_urls

posted = post_replies(MASTODON_INSTANCE, ACCESS_TOKEN, posts_for_llm, reply_batch, visibility="unlisted")


Replied: https://mastodon.social/@cesia/115931320458897718
Replied: https://mastodon.social/@cesia/115931320478007595
Replied: https://mastodon.social/@cesia/115931320537932796
Replied: https://mastodon.social/@cesia/115931320592041289


['https://mastodon.social/@cesia/115931320458897718',
 'https://mastodon.social/@cesia/115931320478007595',
 'https://mastodon.social/@cesia/115931320537932796',
 'https://mastodon.social/@cesia/115931320592041289']